In [1]:
import numpy as np
import pandas as pd
import time
from tqdm.notebook import tqdm
import folium
import json
import googlemaps

In [2]:
tour_info = pd.read_csv('data/관광안내소.csv',thousands=',',encoding='cp949')

In [3]:
tour_info.drop(['데이터기준일자','제공기관코드','제공기관기관명','작업일시',
                '부가서비스정보','평균근무인원수','영어안내가능여부','일본어안내가능여부',
                '중국어안내가능여부','안내가능외국어','안내소위치명','시도명','휴무일',
                '운영기관명','홈페이지주소','소재지지번주소'],axis=1,inplace=True)

In [4]:
f = open("key/명식key.txt", 'r')
gmaps_key =f.readlines()
gmaps = googlemaps.Client(key=gmaps_key[0])

In [5]:
tour_info['안내소소개'].fillna('관광안내소', inplace=True)
tour_info['소재지도로명주소'].fillna('관광안내소', inplace=True)
tour_info['소재지도로명주소'][3] = '서울 은평구 불광동 산42-5'

D:\Users\ryh26\jinho\ipykernel_22764\2262639219.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tour_info['소재지도로명주소'][3] = '서울 은평구 불광동 산42-5'


In [6]:
tour_name = []
for name in tour_info.index:
    tour_name.append(tour_info['관광안내소명'][name])

In [7]:
for n in tour_info.index:
    if tour_info['소재지도로명주소'][n] == '관광안내소':
        tmp = gmaps.geocode(tour_name, language='ko')
        tour_info['소재지도로명주소'][n] = tmp[0].get('formatted_address')
        tmp_loc = tmp[0].get('geometry')
    else: pass

D:\Users\ryh26\jinho\ipykernel_22764\4173704634.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tour_info['소재지도로명주소'][n] = tmp[0].get('formatted_address')


In [8]:
map = folium.Map(location=[37.5502,126.982], zoom_start=11)

for n in tour_info.index:
        folium.Marker(location=[tour_info['위도'][n],tour_info['경도'][n]],
                    tooltip=tour_info['관광안내소명'][n],popup='<pre>'+tour_info['소재지도로명주소'][n]+'</pre>').add_to(map)
map